In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from threading import Thread, RLock
import sys
import time
import os
import random
import matplotlib.pyplot as plt
import re

In [4]:
#Obtention du json pour le paracetamol
url = "https://www.open-medicaments.fr/api/v1/medicaments?limit=100&query=paracetamol"
json = requests.get(url).json()

In [38]:
#Compréhension de liste pour obtenir directement les fiches medicaments
ICS = [f"https://www.open-medicaments.fr/api/v1/medicaments/{elm['codeCIS']}" for elm in json]

#On récupère les json de toutes nos pages medicaments
s=[requests.get(url).json() for url in ICS]

In [33]:
#Pour chaque json, on récupère le nombre de gélulle, disponible dans le libellé
reg1 = r'(\d+)'
libelles = [medoc["presentations"][0]["libelle"] for medoc in s]

#On applique notre regex pour trouver uniquement les valeurs numériques
gelules = pd.DataFrame({"gelules":[re.findall(reg1,lib)[-1] for lib in libelles]})
gelules.head()

,gelules
0,16
1,8
2,8
3,16
4,16


In [34]:
#Exemple d'utilisation de la fonction findall de regex, ici on trouve tout ce qu'il y a après la virgule
string = 'PARACETAMOL ZYDUS 500 mg, gélule'
reg = r', (.*)'
re.findall(reg,string)

['gélule']

In [52]:
#On applique les regex sur un dataframe complet à l'aide de la fonction string

#On creer une df à partir du json de la première recherche
df=pd.DataFrame(json)

#On creer la regex
reg=r'([\D]*)(\d+)(.*),(.*)'

#on isole une série à partir de la colonne denomination du df
serie = df["denomination"]

#On applique la methode str.extract sur notre serie ce qui créer un nouveau df
ds=serie.str.extract(reg)

#On créer une nouvelle colonne qui vaut 1000 par défaut
ds["mul"]=1000

#On applique un masque conditionnel à l'aide de la fonction where, 
#ainsi seuls les lignes dont la colone ds[2] vaut "g" sont modifiés

ds["mul"]=ds["mul"].where(ds[2].str.strip()=="g",1)

#On créer une nouvelle colonne dosage après avoir convertit ds(1) en int et enlevé les valeurs "NaN"
ds["dosage"]=ds[1].fillna(0).astype(int)*ds["mul"]

#On ajoute le nombre de gelules calculés précédemment
ds["nb_gelules"]=gelules
ds.head()

,0,1,2,3,mul,dosage,nb_gelules
0,PARACETAMOL ZYDUS,500,mg,gélule,1,500,16
1,PARACETAMOL RATIOPHARM,1000,mg,comprimé,1,1000,8
2,PARACETAMOL EG,1,g,comprimé,1000,1000,8
3,PARACETAMOL SANDOZ,500,mg,gélule,1,500,16
4,PARACETAMOL RATIOPHARM,500,mg,gélule,1,500,16
